In [1]:

# Install dependencies (requirements.txt) and legal_rag (no deps). Safe for Colab/Jupyter.
import shutil
import subprocess
import sys
from pathlib import Path

REQ_FILE = Path("requirements.txt")
REQ_URL = "https://github.com/artefactory-argimi/legal_rag/releases/download/data-juri-v1/requirements.txt"
REPO_URL = "https://github.com/artefactory-argimi/legal_rag.git"  # change if you fork
UV_BIN = shutil.which("uv")
PIP_CMD = [UV_BIN, "pip"] if UV_BIN else [sys.executable, "-m", "pip"]
FORCE_FLAGS = ["--upgrade", "--force-reinstall"]


def _run(cmd: list[str]) -> None:
    subprocess.check_call(cmd)


# Install full dependencies (with deps) from published requirements, then install legal_rag sans deps.
_run([*PIP_CMD, "install", *FORCE_FLAGS, "-r", REQ_URL])
_run([*PIP_CMD, "install", *FORCE_FLAGS, "--no-deps", f"git+{REPO_URL}"])

# Verify import
import legal_rag as _  # noqa: F401

In [2]:
GENERATOR_API_KEY = ""  # @param {type:"string"}
GENERATOR_API_BASE = ""  # @param {type:"string"}
GENERATOR_MODEL_ID = "Qwen/Qwen3-0.6B"  # @param ["Qwen/Qwen3-0.6B", "mistralai/Mistral-Small-3.1-24B-Instruct-2503"]  # noqa: E501

# Encoder Hugging Face repo id (must match the model used to build the index).
ENCODER_MODEL_ID = "maastrichtlawtech/colbert-legal-french"  # @param {type:"string"}
# Index source built offline from the same encoder. Can be a remote zip or a
# local directory path to a pre-extracted index. INDEX_PATH controls where a zip
# is extracted if a local directory is not passed directly.
INDEX_URI = "https://github.com/artefactory-argimi/legal_rag/releases/download/data-juri-v1/index.zip"  # @param {type:"string"}
INDEX_PATH = "./downloaded/index_legal_constit/"  # @param {type:"string"}
SEARCH_K = 5  # @param {type:"integer"}
MAX_NEW_TOKENS = 512  # @param {type:"integer"}
TEMPERATURE = 0.2  # @param {type:"number"}
MAX_ITERS = 4  # @param {type:"integer"}
INSTRUCTIONS = """Tu es un agent RAG spécialisé en jurisprudence constitutionnelle française (sous-ensemble 'constit' du jeu artefactory/Argimi-Legal-French-Jurisprudence).
Flow obligatoire: (1) search_legal_docs(query="…", k=5) pour obtenir des ids et scores puis (2) lookup_legal_doc(doc_id="…", score=…) sur les ids utiles avant de répondre.
N'emploie jamais de clés args/kwargs ni d'autres paramètres, uniquement query/k ou doc_id/score.
Exemples exacts: search_legal_docs(query="résiliation contrat imprévision", k=5) puis lookup_legal_doc(doc_id="2491", score=4.91).
Toujours réutiliser tel quel le doc_id (string) et le score renvoyés par search_legal_docs.
Si search_legal_docs renvoie "No results.", indique clairement qu'aucune décision pertinente n'a été trouvée.
Si la question dépasse la jurisprudence constitutionnelle (autres branches du droit), explique que le domaine n'est pas couvert et n'appelle pas les outils.
Chaque réponse doit citer explicitement la jurisprudence utilisée (titre ou référence) et la date de la décision.
Présente d'abord les éléments juridiques pertinents (faits, fondement, dispositif, articles cités), puis formule une réponse synthétique.
La réponse doit être une interprétation fondée uniquement sur les décisions récupérées, jamais sur ta mémoire du modèle.
Si aucune décision pertinente n'est récupérée ou si les éléments ne permettent pas de répondre, indique clairement que tu n'as pas les informations nécessaires pour répondre à la question.
Réponds en français de façon précise et utile."""
configured_index = None

"""Run the install cell (demo_install.py) before executing this notebook."""

In [ ]:
import os
from pathlib import Path
from pprint import pformat
from typing import Iterable

from etils import ecolab
from rich import print as rprint

ecolab.auto_display()
ecolab.auto_inspect()
try:
    import google.colab  # type: ignore

    IN_COLAB = True
except Exception:
    IN_COLAB = False

# Avoid optional vision deps when loading text models.
os.environ.setdefault("TRANSFORMERS_NO_TORCHVISION", "1")
os.environ.setdefault("DISABLE_TRANSFORMERS_AV", "1")
from legal_rag.assets import extract_zip, fetch_zip

In [3]:
def prepare_index(index_uri: str | Path, index_dest: Path) -> Path:
    """Fetch and extract the index assets; return the resolved index directory."""
    index_override = Path(index_uri).expanduser()
    index_dest = index_dest.expanduser()
    index_ready = index_dest.exists() and any(index_dest.iterdir())
    if index_override.is_dir():
        index_dir = index_override
    elif index_ready:
        index_dir = index_dest
    else:
        index_dest.mkdir(parents=True, exist_ok=True)
        idx_zip_path = index_dest.parent / "_idx.zip"
        index_zip = fetch_zip(str(index_uri), idx_zip_path)
        try:
            index_dir = extract_zip(index_zip, index_dest)
        finally:
            idx_zip_path.unlink(missing_ok=True)
    fast_plaid_paths = list(index_dir.glob("**/fast_plaid_index"))
    if not fast_plaid_paths:
        raise ValueError(
            f"Invalid index layout at {index_dir}. Expected a fast_plaid_index directory under this folder."
        )
    fast_plaid = sorted(
        fast_plaid_paths, key=lambda p: len(p.relative_to(index_dir).parts)
    )[0]
    index_root = fast_plaid.parent.parent
    if not index_root.exists():
        raise ValueError(
            f"Could not resolve index root from fast_plaid_index at {fast_plaid}"
        )
    return index_root


# Run asset prep early so downstream cells only depend on local paths.
index_source = Path(INDEX_URI).expanduser()
configured_index = prepare_index(index_uri=INDEX_URI, index_dest=Path(INDEX_PATH))
print(f"✓ Encoder will be loaded from Hugging Face repo {ENCODER_MODEL_ID}")
print(f"✓ Index path resolved to {configured_index}")
if index_source.is_dir():
    print(f"✓ Using local index at {configured_index}")
else:
    print(f"✓ Index ready at {configured_index}")

# Increase HF download timeout to reduce transient failures when fetching models.
os.environ.setdefault("HF_HUB_TIMEOUT", "60")

✓ Encoder will be loaded from Hugging Face repo maastrichtlawtech/colbert-legal-french
✓ Index path resolved to downloaded/index_legal_constit
✓ Index ready at downloaded/index_legal_constit


'60'

In [4]:
from legal_rag.agent import build_agent

generator_api_key = GENERATOR_API_KEY or None
generator_api_base = GENERATOR_API_BASE or None

agent = build_agent(
    student_model=GENERATOR_MODEL_ID,
    encoder_model=ENCODER_MODEL_ID,
    generator_api_key=generator_api_key,
    generator_api_base=generator_api_base,
    index_folder=configured_index,  # used by ColBERT retriever in agent.py
    search_k=SEARCH_K,
    max_new_tokens=MAX_NEW_TOKENS,
    temperature=TEMPERATURE,
    instructions=INSTRUCTIONS,
    max_iters=MAX_ITERS,
)

[legal_rag] Loading encoder from maastrichtlawtech/colbert-legal-french


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Tokenizer vocab size: 32007
Embedding layer size: 32007
Query prefix '[Q] ': ID 32006
Document prefix '[D] ': ID 32007

⚠️  Token IDs exceed embedding size!
Resizing from 32007 to 32008
✓ Resized to 32008
✓ All token IDs are now valid
[legal_rag] Loading index from /Users/hicham.randrianarivo/Work/demo_legal_rag/downloaded/index_legal_constit (name=legal_french_index)
[legal_rag] FastPLAID constrained to a single CPU worker.
[legal_rag] PLAID index status is_indexed=True


Resolving data files:   0%|          | 0/35 [00:00<?, ?it/s]

In [5]:
queries: Iterable[str] | str = [
    "Comment obtenir un permis de conduire au Canada ?",  # hors domaine (non constitutionnel français)
    "Quelles sont les conditions de recevabilité d'une question prioritaire de constitutionnalité par le Conseil constitutionnel ?",
]

for idx, question in enumerate(queries, start=1):
    rprint(f"[bold cyan]Question {idx}[/]: {question}")

    prediction = agent(question=question)
    rprint(f"[bold green]Réponse[/]\n{prediction.answer}")
    if IN_COLAB:
        ecolab.json(prediction.trajectory)  # interactive view in Colab
    else:
        rprint(
            f"[bold magenta]Trajectoire[/]\n{pformat(prediction.trajectory, width=100, sort_dicts=False)}"
        )

Question 1: Comment obtenir un permis de conduire au Canada ?

Réponse
La question posée concerne l'obtention d'un permis de conduire au Canada, ce qui ne relève pas de la jurisprudence 
constitutionnelle française. Mon domaine d'expertise est limité à la jurisprudence constitutionnelle française, et 
je ne dispose pas des informations nécessaires pour répondre à cette question.

Trajectoire
{'thought_0': "La question porte sur l'obtention d'un permis de conduire au Canada, ce qui ne "
              'relève pas de la jurisprudence constitutionnelle française. Il est donc nécessaire '
              "d'expliquer que ce domaine n'est pas couvert par l'agent.",
 'tool_name_0': 'finish',
 'tool_args_0': {},
 'observation_0': 'Completed.'}

Question 2: Quelles sont les conditions de recevabilité d'une question prioritaire de constitutionnalité par le 
Conseil constitutionnel ?

Réponse
Les conditions de recevabilité d'une question prioritaire de constitutionnalité (QPC) par le Conseil 
constitutionnel sont définies par plusieurs textes et procédures. Selon la décision relative à la commune du Port 
du 21 octobre 2021, le Conseil constitutionnel est saisi dans les conditions prévues à l'article 61-1 de la 
Constitution. La QPC doit être posée par un justiciable dans le cadre d'une instance en cours devant une 
juridiction. La saisine doit être effectuée par un avocat et enregistrée au secrétariat général du Conseil 
constitutionnel. Les conditions de recevabilité incluent la vérification de la conformité des dispositions 
législatives contestées aux droits et libertés garantis par la Constitution. Le Conseil constitutionnel examine 
ensuite la QPC au regard des textes applicables, tels que la Constitution, l'ordonnance n° 58-1067 du 7 novembre 
1958 portant loi organique sur le Conseil constitutionnel, et le règlement du 4 février 2010 sur la procédure 
suivie devant le Conseil constitutionnel pour les questions prioritaires de constitutionnalité. La décision 
relative à la commune du Port illustre également les étapes procédurales, y compris l'audience publique et les 
observations présentées par les parties.

Trajectoire
{'thought_0': "Pour répondre à la question sur les conditions de recevabilité d'une question "
              'prioritaire de constitutionnalité (QPC) par le Conseil constitutionnel, je dois '
              "d'abord rechercher des décisions pertinentes qui traitent de ce sujet. Je vais "
              'commencer par effectuer une recherche dans les documents juridiques pour obtenir '
              'des références et des scores.',
 'tool_name_0': 'search_legal_docs',
 'tool_args_0': {'query': 'conditions recevabilité question prioritaire constitutionnalité Conseil '
                          'constitutionnel',
                 'k': 5},
 'observation_0': '[0] id=217 score=9.6643\n'
                  '\n'
                  '[1] id=109 score=8.9526\n'
                  '\n'
                  '[2] id=161 score=8.9431\n'
                  '\n'
                  '[3] id=7161 score=8.9065\n'
                  '\n'
                  '[4] id=15 score=8.9065',
 'thought_1': "J'ai obtenu plusieurs références de décisions pertinentes concernant les conditions "
              "de recevabilité d'une question prioritaire de constitutionnalité (QPC) par le "
              'Conseil constitutionnel. Je vais maintenant consulter les détails de ces décisions '
              'pour extraire les informations nécessaires.',
 'tool_name_1': 'lookup_legal_doc',
 'tool_args_1': {'doc_id': '217', 'score': 9.6643},
 'observation_1': 'Document id: 217 (score: 9.6643)\n'
                  "Titre : Commune du Port [Exclusion des communes d'outre-mer de la faculté de "
                  'majorer les indemnités de fonction des élus municipaux des communes '
                  'attributaires de la dotation de solidarité urbaine et de cohésion sociale]\n'
                  'Date : 2021-10-21\n'
                  'Juridiction : \n'
                  'Formation : \n'
                  'Solution : Non conformité totale - effet différé\n'
                  'Droit appliqué : \n'
                  'Texte de la décision : LE CONSEIL CONSTITUTIONNEL A ÉTÉ SAISI le 26 juillet '
                  "2021 par le Conseil d'État (décision n° 452813 du même jour), dans les "
                  "conditions prévues à l'article 61-1 de la Constitution, d'une question "
                  'prioritaire de constitutionnalité. Cette question a été posée pour la commune '
                  'du Port par Me Éric Dugoujon, avocat au barreau de Saint-Denis de La Réunion. '
                  'Elle a été enregistrée au secrétariat général du Conseil constitutionnel sous '
                  'le n° 2021-943 QPC. Elle est relative à la conformité aux droits et libertés '
                  "que la Constitution garantit du 5° de l'article L. 2123-22 du code général des "
                  'collectivités territoriales, dans sa rédaction résultant de la loi n° 2019-1461 '
                  "du 27 décembre 2019 relative à l'engagement dans la vie locale et à la "
                  "proximité de l'action publique.<br>\n"
                  "<br>Au vu des textes suivants :<br>- la Constitution ;<br>- l'ordonnance n° "
                  '58-1067 du 7 novembre 1958 portant loi organique sur le Conseil constitutionnel '
                  ';<br>- le code général des collectivités territoriales ;<br>- la loi n° '
                  "2019-1461 du 27 décembre 2019 relative à l'engagement dans la vie locale et à "
                  "la proximité de l'action publique ;<br>- le règlement du 4 février 2010 sur la "
                  'procédure suivie devant le Conseil constitutionnel pour les questions '
                  'prioritaires de constitutionnalité ;<br>Au vu des pièces suivantes :<br>- les '
                  'observations présentées pour la commune requérante par Me Dugoujon, '
                  'enregistrées le 17 août 2021 ;<br>- les observations présentées par le Premier '
                  'ministre, enregistrées le même jour ;<br>- les autres pièces produites et '
                  'jointes 